# MMU metrics walkthrough

This notebook briefly demonstrates the various capabilities of the package on the computation of confusion matrix/matrices and binary classification metrics.

In [1]:
import pandas as pd
import numpy as np

import mmu

### Data generation

We generate predictions and true labels where:
* `proba`: estimated probabilities
* `yhat`: estimated labels
* `y`: true labels

In [2]:
proba, yhat, y = mmu.generate_data(n_samples=10000)

### Confusion matrix only

We can compute the confusion matrix for a single run using the estimated labels or based on the probability and a classification threshold.

Based on the esstimated labels `yhat`

In [3]:
# based on yhat
mmu.confusion_matrix(y, yhat)

array([[ 843, 2865],
       [1414, 4878]])

based on proba with classification threshold

In [4]:
mmu.confusion_matrix_proba(y, proba, threshold=0.5)

array([[ 843, 2865],
       [1414, 4878]])

## Confusion matrix and metrics

The ``binary_metrics*`` functions compute ten classification metrics:
 *    0 - neg.precision aka Negative Predictive Value
 *    1 - pos.precision aka Positive Predictive Value
 *    2 - neg.recall aka True Negative Rate & Specificity
 *    3 - pos.recall aka True Positive Rate aka Sensitivity
 *    4 - neg.f1 score
 *    5 - pos.f1 score
 *    6 - False Positive Rate
 *    7 - False Negative Rate
 *    8 - Accuracy
 *    9 - MCC

These metrics were chosen as they are the most commonly used metrics and most other metrics can be compute from these. We don't provide individual functions at the moment as the overhead of computing all of them vs one or two is negligable.

This index can be retrieved using:

In [5]:
col_index = mmu.metrics.col_index
col_index

{'neg.precision': 0,
 'neg.prec': 0,
 'npv': 0,
 'pos.precision': 1,
 'pos.prec': 1,
 'ppv': 1,
 'neg.recall': 2,
 'neg.rec': 2,
 'tnr': 2,
 'specificity': 2,
 'pos.recall': 3,
 'pos.rec': 3,
 'tpr': 3,
 'sensitivity': 3,
 'neg.f1': 4,
 'neg.f1_score': 4,
 'pos.f1': 5,
 'pos.f1_score': 5,
 'fpr': 6,
 'fnr': 7,
 'accuracy': 8,
 'acc': 8,
 'mcc': 9}

### For a single test set

In [6]:
cm, metrics = mmu.binary_metrics(y, yhat)

In [7]:
# the confusion matrix
cm

array([[ 843, 2865],
       [1414, 4878]])

We can create a dataframe from the confusion matrix using:

In [8]:
mmu.confusion_matrix_to_dataframe(cm)

estimated         
                   negative positive
observed negative       843     2865
         positive      1414     4878

In [9]:
metrics

array([0.37350465, 0.62998838, 0.22734628, 0.77527018, 0.28264878,
       0.69511934, 0.77265372, 0.22472982, 0.5721    , 0.00302314])

We can create a dataframe from the metrics using:

In [10]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,0.373505,0.629988,0.227346,0.77527,0.282649,0.695119,0.772654,0.22473,0.5721,0.003023


### A single run using probabilities

In [11]:
cm, metrics = mmu.binary_metrics_proba(y, proba, threshold=0.5)

In [12]:
mmu.confusion_matrix_to_dataframe(cm)

estimated         
                   negative positive
observed negative       843     2865
         positive      1414     4878

In [13]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,0.373505,0.629988,0.227346,0.77527,0.282649,0.695119,0.772654,0.22473,0.5721,0.003023


### A single run using multiple thresholds

Can be used when you want to compute a precision-recall curve for example

In [14]:
thresholds = np.linspace(1e-5, 1.0, 1000)

In [15]:
cm, metrics = mmu.binary_metrics_thresholds(
    y=y,
    proba=proba,
    thresholds=thresholds,
)

The confusion matrix is now an 2D array where the rows contain the confusion matrix for a single threshold

In [16]:
cm

array([[   0, 3708,    0, 6292],
       [   0, 3708,    0, 6292],
       [   0, 3708,    0, 6292],
       ...,
       [3708,    0, 6292,    0],
       [3708,    0, 6292,    0],
       [3708,    0, 6292,    0]])

Similarly, `metrics` is now an 2D array where the rows contain the metrics for a single threshold

In [17]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,0.0000,0.6292,0.0,1.0,0.000000,0.772404,1.0,0.0,0.6292,0.0
1,0.0000,0.6292,0.0,1.0,0.000000,0.772404,1.0,0.0,0.6292,0.0
2,0.0000,0.6292,0.0,1.0,0.000000,0.772404,1.0,0.0,0.6292,0.0
3,0.0000,0.6292,0.0,1.0,0.000000,0.772404,1.0,0.0,0.6292,0.0
4,0.0000,0.6292,0.0,1.0,0.000000,0.772404,1.0,0.0,0.6292,0.0
...,...,...,...,...,...,...,...,...,...,...
995,0.3708,0.0000,1.0,0.0,0.540998,0.000000,0.0,1.0,0.3708,0.0
996,0.3708,0.0000,1.0,0.0,0.540998,0.000000,0.0,1.0,0.3708,0.0
997,0.3708,0.0000,1.0,0.0,0.540998,0.000000,0.0,1.0,0.3708,0.0
998,0.3708,0.0000,1.0,0.0,0.540998,0.000000,0.0,1.0,0.3708,0.0


Generate multiple runs for the below functions

In [18]:
proba, yhat, y = mmu.generate_data(n_samples=10000, n_sets=100)

### Multiple runs using a single threshold

You have performed bootstrap or multiple train-test runs and want to evaluate the distribution of the metrics you can use `binary_metrics_runs`.

`cm` and `metrics` are now two dimensional arrays where the rows are the confusion matrices/metrics for that a run

In [19]:
cm, metrics = mmu.binary_metrics_runs(
    y=y,
    proba=proba,
    threshold=0.5,
)

In [20]:
cm[:5, :]

array([[ 853, 2879, 1413, 4855],
       [ 839, 2843, 1501, 4817],
       [ 806, 2923, 1429, 4842],
       [ 837, 2958, 1389, 4816],
       [ 862, 2907, 1416, 4815]])

### Multiple runs using multiple thresholds

You have performed bootstrap or multiple train-test runs and, for example, want to evaluate the different precision recall curves

In [21]:
cm, metrics = mmu.binary_metrics_runs_thresholds(
    y=y,
    proba=proba,
    thresholds=thresholds,
    fill=1.0
)

The confusion matrix and metrics are now cubes.

For the confusion matrix the:
* row -- the thresholds
* colomns -- the confusion matrix elements
* slices -- the runs

For the metrics:

* row -- thresholds
* colomns -- the metrics
* slices -- the runs

The stride is such that the biggest stride is over the thresholds for the confusion matrix and over the metrics for the metrics.

The argument being that you will want to model the confusion matrices over the runs
and the metrics individually over the thresholds and runs

In [22]:
print('shape confusion matrix: ', cm.shape)
print('strides confusion matrix: ', cm.strides)

shape confusion matrix:  (1000, 4, 100)
strides confusion matrix:  (3200, 8, 32)


In [23]:
print('shape metrics: ', metrics.shape)
print('strides metrics: ', metrics.strides)

shape metrics:  (1000, 10, 100)
strides metrics:  (800, 800000, 8)


In [24]:
pos_recalls = metrics[:, mmu.metrics.col_index['pos.rec'], :]
pos_precisions = metrics[:, mmu.metrics.col_index['pos.prec'], :]

### Binary metrics over confusion matrices

This can be used when you have a methodology where you model and generate confusion matrices

In [25]:
# We use binary_metrics_runs to create confusion matrices
cm, _ = mmu.binary_metrics_runs(
    y=y,
    proba=proba,
    threshold=0.5,
    fill=0.0
)

In [26]:
metrics = mmu.binary_metrics_confusion(cm, 0.0)

In [27]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,0.376434,0.627748,0.228564,0.774569,0.284428,0.693472,0.771436,0.225431,0.5708,0.003620
1,0.358547,0.628851,0.227865,0.762425,0.278645,0.689226,0.772135,0.237575,0.5656,-0.011062
2,0.360626,0.623567,0.216144,0.772126,0.270288,0.689940,0.783856,0.227874,0.5648,-0.013617
3,0.376011,0.619501,0.220553,0.776148,0.278027,0.689034,0.779447,0.223852,0.5653,-0.003848
4,0.378402,0.623543,0.228708,0.772749,0.285100,0.690174,0.771292,0.227251,0.5677,0.001684
...,...,...,...,...,...,...,...,...,...,...
95,0.369032,0.612508,0.223904,0.762160,0.278707,0.679188,0.776096,0.237840,0.5559,-0.016039
96,0.376991,0.620413,0.224802,0.773269,0.281653,0.688459,0.775198,0.226731,0.5654,-0.002238
97,0.382114,0.623940,0.224165,0.780276,0.282565,0.693406,0.775835,0.219724,0.5704,0.005186
98,0.375329,0.630148,0.230396,0.773732,0.285523,0.694597,0.769604,0.226268,0.5721,0.004755
